G1 Project 3

Natural Language Processing Challenge

Introduction
Learning how to process text is a skill required for Data Scientists. In this project, you will put these skills into practice to identify whether a sentence was automatically translated or translated by a human.

Project Overview
In this repository you will find dataset containing sentences in Spanish and their tags: 0, if the sentences was translated by a Machine, 1, if the sentence was translated by a professional translator. Your goal is to build a classifier that is able to distinguish between the two.

Guidance
Like in a real life scenario, you are able to make your own choices and text treatment. Use the techniques you have learned and the common packages to process this data and classify the text.

Deliverables
Python Code: Provide well-documented Python code that conducts the analysis.
Accuracy estimation: Provide the teacher with your estimation of how your model will perform.
Classified Dataset: On Friday, you will receive a dataset without tags. Prepare your code to be able to tag that dataset.

Import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

1. Download DATA

In [ ]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

2. Dividing Training and Test Set

In [ ]:
from sklearn.model_selection import train_test_split

3. Data Preprocessing

4.  Removing Words

5. Tame Your Text with Lemmatization

6. Bag Of Words (*)

7. Extra features

8. How would Bag of Words with Count Vectorizer work?

9. TF-IDF (*)

10.  Train a Classifier